In [1]:
%load_ext autoreload
%autoreload 2

import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="3" #! specify gpu here
import sys
import numpy as np
import cv2
import time
import torch
import clip
from PIL import Image
from rich import print
from tqdm import tqdm
from pathlib import Path
import csv
import json
import click


# for jsonpickle
from context_action_framework.types import Detection, Label, LabelFace, Camera
from geometry_msgs.msg import Transform, Vector3, Quaternion, Pose, PoseArray, TransformStamped

import jsonpickle
import pickle

from context_action_framework.types import detections_to_ros, detections_to_py
from vision_pipeline.llm_data_generator.labelme_importer import LabelMeImporter

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
labelme_importer = LabelMeImporter()

# from IPython.utils import io

# with io.capture_output() as captured:

config:
{
    'node_name': 'vision',
    'reid': False,
    'basler': {
        'target_fps': 10,
        'max_allowed_acquisition_delay': 1.0,
        'rotate_img': 180,
        'topic': 'basler',
        'camera_node': '/basler',
        'image_topic': 'image_rect_color',
        'sleep_camera_on_exit': False,
        'publish_labelled_img': True,
        'publish_graph_img': True,
        'has_depth': False,
        'run_continuous': True,
        'wait_for_services': False,
        'detect_arucos': False,
        'table_name': 'table_vision',
        'parent_frame': 'vision_table_zero',
        'create_parent_frame': True,
        'marker_lifetime': 1,
        'work_surface_ignore_border_width': 50,
        'show_work_surface_detection': False,
        'use_worksurface_detection': True,
        'debug_work_surface_detection': False
    },
    'realsense': {
        'target_fps': 10,
        'max_allowed_acquisition_delay': 1.0,
        'rotate_img': 0,
        'topic': 'realsense',
        'camera_node': '/realsense',
        'image_topic': 'color/image_raw',
        'info_topic': 'color/camera_info',
        'depth_topic': 'aligned_depth_to_color/image_raw',
        'sleep_camera_on_exit': False,
        'publish_labelled_img': True,
        'publish_depth_img': True,
        'publish_cluster_img': True,
        'publish_graph_img': False,
        'has_depth': True,
        'compute_gaps': True,
        'run_continuous': True,
        'wait_for_services': False,
        'detect_arucos': False,
        'camera_height': 0.2,
        'parent_frame': 'panda_2/realsense',
        'create_parent_frame': True,
        'marker_lifetime': 1,
        'calibration_file': '~/vision_pipeline/realsense_calib/realsense_calib.yaml',
        'debug_clustering': False
    },
    'obj_detection': {
        'debug': True,
        'model': 'yolov8',
        'yolact_dataset_file': '~/vision_pipeline/data_limited/yolact/2023-07-18_firealarms_hcas/dataset.json',
        'yolact_score_threshold': 0.5,
        'yolov8_model_file': 
'~/vision_pipeline/data_limited/yolov8/output_2024-07-17_20000_incl_new_jsi_imgs_p2/epoch60.pt',
        'yolov8_score_threshold': 0.5,
        'superglue_templates': '~/datasets2/reconcycle/2023-12-04_hcas_fire_alarms_sorted_cropped',
        'superglue_model_file': 
'~/superglue_training/output/train/2024-06-26_superglue_model_evens_finished/weights/best.pt',
        'superglue_match_threshold': 0.5,
        'rotation_median_filter': False,
        'superglue_visualise_to_file': False,
        'classifier_model_file': 
'~/vision_pipeline/data_limited/classifier/2024-07-19__14-32_classify/lightning_logs/version_0/checkpoints/epoch=33
9-step=339.ckpt',
        'classifier_threshold': 0.2
    }
}

/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


model.learning_rate 1e-05

model.batch_size 64

model.freeze_backbone False

curr_weights_path 
/home/docker/superglue_training/output/train/2024-06-26_superglue_model_evens_finished/weights/best.pt

Loaded SuperPoint model
Loaded SuperGlue model ("/home/docker/superglue_training/output/train/2024-06-26_superglue_model_evens_finished/weights/best.pt" weights)


100%|██████████| 61/61 [00:01<00:00, 48.71it/s]


Loaded superglue templates 61

In [7]:
# folder = kg_nodes_path / "kalo"
# folder = kg_nodes_path / "hekatron"
# folder = kg_nodes_path / "qundis"
folder = Path("~/datasets2/reconcycle/knowledge_tree_tests").expanduser()
    
# gt_device = folder.name
# print("gt_device", gt_device)

for img_path, process_labelme_dir, detections, graph_relations, module, camera, batch_crop_imgs in labelme_importer.process_labelme_dir(folder, use_yield=True):

    stem = Path(img_path).stem


    A_relations = graph_relations.to_text()
    print("A_relations:", A_relations)
    
    # include the module name in the relations
    if module is not None:
        print("module", module.name)
        A_relations = f"at_location(table_{module.name}). " + A_relations
    else:
        print("[red]Missing module name!")
    
    positions_list = []
    for detection in detections:
        text = f"{detection.label.name}"
        if detection.label_face is not None:
            text += f" {detection.label_face.name}"
        
        # todo: better label names
        # if detection.label_precise is not None:
        #     label_id = detection.label_precise.split("_")[-1]
        #     text += f" (id: {label_id})"

        if detection.label_precise_name is not None:
            text += f" ({detection.label_precise_name})"
    
        text += f" at ({detection.center_px[0]}, {detection.center_px[1]})"
        positions_list.append(text)

        # print("detection name", detection.label.name)
        # print(f"detection center: {detection.center_px[0]}, {detection.center_px[1]}")

        # print("detection", detection)
    
    A_positions = ", ".join(positions_list) + ". "

    print("A_positions:", A_positions)
    

    


resetting worksurface detection

Converting 00_template_0010:   0%|          | 0/4 [00:00<?, ?it/s]

filename 00_template_0010.json

running worksurface detection...

camera: None

...

Missing module name!

A_positions: hca back (kalo2) at (1434, 1309), battery at (1337, 1308), pcb at (1509, 1311).

resetting worksurface detection

Converting 00_template_0073: 100%|██████████| 4/4 [00:00<00:00,  4.52it/s]


shutdown request: [/asdf] Reason: new node registered with same name
